In [1]:
import fileio
import pydata

#import numpy as np
#import pandas as pd

In [27]:
from importlib import reload

In [164]:
reload(fileio)
#reload(shapes)

<module 'fileio' from '/home/nelson/Data/kba/fileio.py'>

In [2]:
myfile = fileio.InputData('arecaceae.csv')
myfile.iucnFile('arecaceae_categories.csv')

In [ ]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [ ]:
myar.spp_inclusion(myfile)

In [ ]:
myar.new_spp_table('test_kba_log.csv')

---

In [3]:
tilas = myfile.getTiles(1, offsetLat=0.2, offsetLon=0.2)

In [4]:
for ti in tilas:
    print(myfile.tile2str(ti))

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - 0 0 - - - 
- - - - - - 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 1 0 0 0 - - - 
- - - 0 0 1 1 1 0 0 - - 
- - - - 0 0 1 1 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 0 - - - 
- - - - 1 0 - 0 - - - - 
- - 0 0 0 0 0 - - - - - 
- - 0 0 0 - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - 0 0 - - - 
- - - - - - 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 1 0 1 1 - - 
- - - - 0 0 1 0 1 - - - 
- - - - 0 0 1 1 1 0 - - 
- - - - 0 0 0 1 0 - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 1 1 - - - - - 
- - 0 0 1 - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - 0 0 - - - 
- - - - - - 1 0 0 - - -

In [18]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 100, 1)

In [46]:
from shapely.geometry import shape, Polygon, MultiPolygon, mapping, Point
from shapely.ops import unary_union 

import fiona
from fiona.crs import from_epsg

In [26]:
import os

In [33]:
import shapes

In [34]:
shapes.solution2shape(mysols, myfile)

In [41]:
procAreas = {}

filehandle = fiona.open('/home/nelson/Data/GIS/Areas_protegidas/RUNAP/runap2Polygon.shp', crs= 'EPSG:4326', encoding = 'utf8')
for item in filehandle:
    #self.polys.append(shape(item['geometry']))
    procAreas[item['properties']['id_pnn']] = {
        'shape': shape(item['geometry'])
        }

In [45]:
procAreas['20170004']['shape'].contains

<bound method BaseGeometry.contains of <shapely.geometry.polygon.Polygon object at 0x7f24837f9d90>>

In [50]:
polys = []
for ic in range(mysols[1][0].getSize()):
    if mysols[1][0].getValue(ic) > 0:
        y, x = irkeys[ic]
        xBase = myfile.originN[0] + myfile.cellSize * x
        yBase = myfile.originN[1] - myfile.cellSize * y
        ocor = [(xBase + myfile.cellSize, yBase),
            (xBase, yBase),
            (xBase, yBase - myfile.cellSize),
            (xBase + myfile.cellSize, yBase - myfile.cellSize)]
        polys.append(Polygon(ocor))
solpoly = unary_union(polys)

for sp in myfile.points:
    for lon, lat in myfile.points[sp]:
        if solpoly.contains(Point(lon, lat)):
            print (sp)


Wettinia hirsuta
Wettinia hirsuta
Wettinia hirsuta
Wettinia hirsuta
Wettinia hirsuta
Wettinia hirsuta
Wettinia hirsuta
Aiphanes lindeniana
Aiphanes lindeniana
Aiphanes lindeniana
Aiphanes lindeniana
Aiphanes lindeniana
Aiphanes lindeniana
Astrocaryum malybo
Astrocaryum malybo
Astrocaryum malybo
Astrocaryum malybo
Ceroxylon vogelianum
Ceroxylon vogelianum
Chamaedorea ricardoi
Chamaedorea ricardoi
Chamaedorea ricardoi
Chamaedorea ricardoi
Chamaedorea ricardoi
Chamaedorea ricardoi
Chamaedorea ricardoi
Elaeis oleifera
Aiphanes leiostachys
Aiphanes leiostachys
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Aiphanes parvifolia
Astrocaryum triandrum
Astrocaryum triandrum
Astrocaryum triandrum
Astrocaryum triandrum
Attalea cohune
Bactris gasipaes
Ceroxylon quindiuense
Ceroxylon quindiuense
Ceroxylon sasaimae
Cryosophila kalbreyeri
Cryosophila 

In [48]:
solpoly.contains

<bound method BaseGeometry.contains of <shapely.geometry.polygon.Polygon object at 0x7f2483775810>>

In [25]:
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int',
                  'score': 'float',
                  'NDMscore': 'float'},
    }

irkeys = list(myfile.index_reg.keys())

for igr, gr in enumerate(mysols):
    solpolcoll = []
    multipol = None
    filename = 'group_{0}.shp'.format(igr)

    for its,  tsol in enumerate(gr):
        polys = []
        solpoly = None
        for ic in range(tsol.getSize()):
            if tsol.getValue(ic) > 0:
                y, x = irkeys[ic]
                xBase = myfile.originN[0] + myfile.cellSize * x
                yBase = myfile.originN[1] - myfile.cellSize * y
                ocor = [(xBase + myfile.cellSize, yBase),
                    (xBase, yBase),
                    (xBase, yBase - myfile.cellSize),
                    (xBase + myfile.cellSize, yBase - myfile.cellSize)]
                polys.append(Polygon(ocor))
        solpoly = unary_union(polys)
        
        if its == 0:
                with fiona.open(filename, 'w', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                    c.write({'geometry': mapping(solpoly),
                        'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                            })

        else:
            with fiona.open(filename, 'a', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                c.write({'geometry': mapping(solpoly),
                    'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                        })


In [31]:
for spp in [tilas[x].getName() for x in mysols[0][0].spp2crit.keys()]:
    print(spp, len(myfile.points[spp]))

Wettinia hirsuta 16
Aiphanes lindeniana 31
Ceroxylon vogelianum 24
Chamaedorea ricardoi 11
Aiphanes leiostachys 2
Aiphanes parvifolia 11
Astrocaryum triandrum 6
Attalea cohune 2
Attalea nucifera 15
Bactris gasipaes 24
Ceroxylon quindiuense 30
Ceroxylon sasaimae 6
Cryosophila kalbreyeri 27
Geonoma bernalii 2
Geonoma chlamydostachys 12
Geonoma galeanoae 5
Hyospathe wendlandiana 5
Pholidostachys sanluisensis 4


In [26]:
print(myfile.tile2str( mysols[1][0]))

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 1 0 0 0 - - 
- - - - 0 0 1 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 



In [6]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 500, 2)

for ig, group in enumerate(mysols):
    print ('#' * 50)
    print ('Group {0}'.format(ig))
    for iso, so in enumerate(group):
        print("Solution {0} of {1}".format(iso, len(group)))
        print(so.score, "-", so.ndmScore, "-", so.aggrScore)
        #print("Islands: ", pydata.islNum(so))
        #if not pydata.isCont(so):
        #    print("NOT CONTINUOUS!!!")
        #print(so.toBitList())
        print(myfile.tile2str(so))
    

##################################################
Group 0
Solution 0 of 40
18 - 7.264217488467693 - 32.52843497693539
0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 0 0 0 0 - - 
- - - - 0 0 1 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

Solution 1 of 40
8 - 3.9962733779102564 - 15.992546755820513
0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 1 0 0 0 0 - - - 
- - - 0 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - -